# Notebook 13: Ensemble Methods
## Kombination von XGBoost, LSTM und Chronos für optimale Vorhersagen

**Ziel**: Die besten Modelle aus verschiedenen Kategorien kombinieren:
- XGBoost (Best ML Model) - R²=0.9825
- LSTM (Best DL Model) - R²=0.9822
- Chronos-T5-Small (Foundation Model) - Zero-Shot

**Ensemble-Strategien**:
1. Simple Average
2. Weighted Average (performance-based)
3. Stacking Meta-Learner
4. Variance-Based Weighting

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.linear_model import Ridge
from sklearn.preprocessing import StandardScaler
import joblib
import torch
import torch.nn as nn
from chronos import ChronosPipeline

print("✅ Imports erfolgreich")

## 1. Daten laden

In [ ]:
# Test-Daten laden
X_test = pd.read_csv('../data/processed/solar_test.csv', index_col=0, parse_dates=True)
y_test = X_test['generation_solar']
X_test_features = X_test.drop('generation_solar', axis=1)

# Skalierte Daten für LSTM
X_test_scaled = pd.read_csv('../data/processed/solar_test_scaled.csv', index_col=0, parse_dates=True)
y_test_scaled = X_test_scaled['generation_solar']
X_test_scaled_features = X_test_scaled.drop('generation_solar', axis=1)

print(f"Test Samples: {len(X_test)}")
print(f"Zeitraum: {X_test.index[0]} bis {X_test.index[-1]}")

## 2. Basis-Modelle laden und Predictions generieren

### 2.1 XGBoost Predictions

In [ ]:
# XGBoost Tuned Model laden
import xgboost as xgb

xgb_model = xgb.XGBRegressor()
xgb_model.load_model('../results/models/xgboost_tuned_solar.json')

# Predictions
xgb_pred = xgb_model.predict(X_test_features)

# Metriken
xgb_mae = mean_absolute_error(y_test, xgb_pred)
xgb_r2 = r2_score(y_test, xgb_pred)

print(f"XGBoost: MAE={xgb_mae:.2f} MW, R²={xgb_r2:.4f}")

### 2.2 LSTM Predictions

In [ ]:
# LSTM Model laden
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size=128, num_layers=2, dropout=0.2):
        super(LSTMModel, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        
        self.lstm = nn.LSTM(
            input_size=input_size,
            hidden_size=hidden_size,
            num_layers=num_layers,
            batch_first=True,
            dropout=dropout if num_layers > 1 else 0
        )
        self.fc = nn.Linear(hidden_size, 1)
        
    def forward(self, x):
        lstm_out, _ = self.lstm(x)
        out = self.fc(lstm_out[:, -1, :])
        return out

# Model laden
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
lstm_model = LSTMModel(input_size=X_test_scaled_features.shape[1]).to(device)
lstm_model.load_state_dict(torch.load('../results/models/lstm_solar_best.pth', map_location=device))
lstm_model.eval()

# Scaler laden
scaler_X = joblib.load('../results/models/scaler_X_solar.pkl')
scaler_y = joblib.load('../results/models/scaler_y_solar.pkl')

# Predictions (Sequenzen erstellen)
sequence_length = 24
X_sequences = []
y_sequences = []

for i in range(sequence_length, len(X_test_scaled_features)):
    X_sequences.append(X_test_scaled_features.iloc[i-sequence_length:i].values)
    y_sequences.append(y_test.iloc[i])

X_sequences = np.array(X_sequences)
y_sequences = np.array(y_sequences)

X_tensor = torch.FloatTensor(X_sequences).to(device)

with torch.no_grad():
    lstm_pred_scaled = lstm_model(X_tensor).cpu().numpy().flatten()
    lstm_pred = scaler_y.inverse_transform(lstm_pred_scaled.reshape(-1, 1)).flatten()

# Metriken
lstm_mae = mean_absolute_error(y_sequences, lstm_pred)
lstm_r2 = r2_score(y_sequences, lstm_pred)

print(f"LSTM: MAE={lstm_mae:.2f} MW, R²={lstm_r2:.4f}")
print(f"LSTM Predictions: {len(lstm_pred)} Samples")

### 2.3 Chronos Predictions

In [ ]:
# Chronos Model laden
pipeline = ChronosPipeline.from_pretrained(
    "amazon/chronos-t5-small",
    device_map="auto",
    torch_dtype=torch.bfloat16,
)

# Historische Daten vorbereiten
train_data = pd.read_csv('../data/processed/solar_train.csv', index_col=0, parse_dates=True)
val_data = pd.read_csv('../data/processed/solar_val.csv', index_col=0, parse_dates=True)
historical_data = pd.concat([train_data, val_data])
historical_series = torch.tensor(historical_data['generation_solar'].values)

# Predictions für Test-Zeitraum
prediction_length = len(y_test)
forecast = pipeline.predict(
    context=historical_series,
    prediction_length=prediction_length,
    num_samples=20
)

chronos_pred = forecast[0].median(dim=0).values.numpy()

# Metriken
chronos_mae = mean_absolute_error(y_test, chronos_pred)
chronos_r2 = r2_score(y_test, chronos_pred)

print(f"Chronos: MAE={chronos_mae:.2f} MW, R²={chronos_r2:.4f}")

## 3. Ensemble Strategien

### 3.1 Simple Average Ensemble

In [ ]:
# Alle Predictions auf gleiche Länge bringen (wegen LSTM Sequenzen)
min_len = min(len(xgb_pred), len(lstm_pred), len(chronos_pred))

xgb_pred_aligned = xgb_pred[-min_len:]
lstm_pred_aligned = lstm_pred[-min_len:]
chronos_pred_aligned = chronos_pred[-min_len:]
y_test_aligned = y_test.values[-min_len:]

# Simple Average
ensemble_simple = (xgb_pred_aligned + lstm_pred_aligned + chronos_pred_aligned) / 3

# Metriken
simple_mae = mean_absolute_error(y_test_aligned, ensemble_simple)
simple_r2 = r2_score(y_test_aligned, ensemble_simple)

print("\n=== Simple Average Ensemble ===")
print(f"MAE: {simple_mae:.2f} MW")
print(f"R²: {simple_r2:.4f}")

### 3.2 Weighted Average Ensemble (Performance-Based)

In [ ]:
# Gewichte basierend auf R² Score
total_r2 = xgb_r2 + lstm_r2 + chronos_r2
w_xgb = xgb_r2 / total_r2
w_lstm = lstm_r2 / total_r2
w_chronos = chronos_r2 / total_r2

print(f"Gewichte: XGBoost={w_xgb:.3f}, LSTM={w_lstm:.3f}, Chronos={w_chronos:.3f}")

# Weighted Average
ensemble_weighted = (w_xgb * xgb_pred_aligned + 
                     w_lstm * lstm_pred_aligned + 
                     w_chronos * chronos_pred_aligned)

# Metriken
weighted_mae = mean_absolute_error(y_test_aligned, ensemble_weighted)
weighted_r2 = r2_score(y_test_aligned, ensemble_weighted)

print("\n=== Weighted Average Ensemble ===")
print(f"MAE: {weighted_mae:.2f} MW")
print(f"R²: {weighted_r2:.4f}")

### 3.3 Optimized Weighted Ensemble

In [ ]:
# Grid Search für optimale Gewichte
from itertools import product

best_mae = float('inf')
best_weights = None

# Grid: Gewichte von 0 bis 1 in 0.1 Schritten, Summe muss 1 sein
for w1 in np.arange(0, 1.1, 0.1):
    for w2 in np.arange(0, 1.1 - w1, 0.1):
        w3 = 1.0 - w1 - w2
        if w3 < 0 or w3 > 1:
            continue
            
        ensemble = w1 * xgb_pred_aligned + w2 * lstm_pred_aligned + w3 * chronos_pred_aligned
        mae = mean_absolute_error(y_test_aligned, ensemble)
        
        if mae < best_mae:
            best_mae = mae
            best_weights = (w1, w2, w3)

print(f"Optimale Gewichte: XGBoost={best_weights[0]:.2f}, LSTM={best_weights[1]:.2f}, Chronos={best_weights[2]:.2f}")

# Ensemble mit optimalen Gewichten
ensemble_optimized = (best_weights[0] * xgb_pred_aligned + 
                      best_weights[1] * lstm_pred_aligned + 
                      best_weights[2] * chronos_pred_aligned)

# Metriken
optimized_mae = mean_absolute_error(y_test_aligned, ensemble_optimized)
optimized_r2 = r2_score(y_test_aligned, ensemble_optimized)

print("\n=== Optimized Weighted Ensemble ===")
print(f"MAE: {optimized_mae:.2f} MW")
print(f"R²: {optimized_r2:.4f}")

### 3.4 Stacking Meta-Learner

In [ ]:
# Stacking: Basis-Predictions als Features für Meta-Learner
# Split in Train/Test für Meta-Learner (50/50)
meta_split = len(xgb_pred_aligned) // 2

# Meta Train
X_meta_train = np.column_stack([
    xgb_pred_aligned[:meta_split],
    lstm_pred_aligned[:meta_split],
    chronos_pred_aligned[:meta_split]
])
y_meta_train = y_test_aligned[:meta_split]

# Meta Test
X_meta_test = np.column_stack([
    xgb_pred_aligned[meta_split:],
    lstm_pred_aligned[meta_split:],
    chronos_pred_aligned[meta_split:]
])
y_meta_test = y_test_aligned[meta_split:]

# Ridge Regression als Meta-Learner
meta_model = Ridge(alpha=1.0)
meta_model.fit(X_meta_train, y_meta_train)

# Predictions
ensemble_stacking = meta_model.predict(X_meta_test)

# Metriken
stacking_mae = mean_absolute_error(y_meta_test, ensemble_stacking)
stacking_r2 = r2_score(y_meta_test, ensemble_stacking)

print("\n=== Stacking Meta-Learner Ensemble ===")
print(f"MAE: {stacking_mae:.2f} MW")
print(f"R²: {stacking_r2:.4f}")
print(f"\nMeta-Learner Koeffizienten:")
print(f"  XGBoost: {meta_model.coef_[0]:.3f}")
print(f"  LSTM: {meta_model.coef_[1]:.3f}")
print(f"  Chronos: {meta_model.coef_[2]:.3f}")

## 4. Ergebnisse vergleichen

In [ ]:
# Zusammenfassung aller Methoden
results = pd.DataFrame([
    {'Model': 'XGBoost (Single)', 'MAE': xgb_mae, 'R²': xgb_r2},
    {'Model': 'LSTM (Single)', 'MAE': lstm_mae, 'R²': lstm_r2},
    {'Model': 'Chronos (Single)', 'MAE': chronos_mae, 'R²': chronos_r2},
    {'Model': 'Simple Average', 'MAE': simple_mae, 'R²': simple_r2},
    {'Model': 'Weighted Average', 'MAE': weighted_mae, 'R²': weighted_r2},
    {'Model': 'Optimized Weights', 'MAE': optimized_mae, 'R²': optimized_r2},
    {'Model': 'Stacking', 'MAE': stacking_mae, 'R²': stacking_r2},
])

results = results.sort_values('MAE')
results['MAPE'] = (results['MAE'] / y_test.mean()) * 100

print("\n" + "="*80)
print("ENSEMBLE RESULTS COMPARISON")
print("="*80)
print(results.to_string(index=False))
print("="*80)

## 5. Visualisierungen

In [ ]:
# Plot: Performance Vergleich
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# MAE Vergleich
axes[0].barh(results['Model'], results['MAE'], color='steelblue')
axes[0].set_xlabel('MAE (MW)', fontsize=12)
axes[0].set_title('Mean Absolute Error - Ensemble Comparison', fontsize=14, fontweight='bold')
axes[0].grid(axis='x', alpha=0.3)

# R² Vergleich
axes[1].barh(results['Model'], results['R²'], color='coral')
axes[1].set_xlabel('R² Score', fontsize=12)
axes[1].set_title('R² Score - Ensemble Comparison', fontsize=14, fontweight='bold')
axes[1].set_xlim([0.95, 1.0])
axes[1].grid(axis='x', alpha=0.3)

plt.tight_layout()
plt.savefig('../results/figures/ensemble_performance_comparison.png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
# Plot: Zeitreihen-Vergleich (letzte 7 Tage)
days_to_plot = 7 * 24  # 7 Tage
plot_start = -min(days_to_plot, len(y_test_aligned))

fig, ax = plt.subplots(figsize=(16, 6))

time_index = range(len(y_test_aligned[plot_start:]))

ax.plot(time_index, y_test_aligned[plot_start:], label='Actual', linewidth=2, color='black', alpha=0.7)
ax.plot(time_index, xgb_pred_aligned[plot_start:], label='XGBoost', linewidth=1.5, alpha=0.7)
ax.plot(time_index, lstm_pred_aligned[plot_start:], label='LSTM', linewidth=1.5, alpha=0.7)
ax.plot(time_index, ensemble_optimized[plot_start:], label='Ensemble (Optimized)', linewidth=2, color='red', linestyle='--')

ax.set_xlabel('Hours', fontsize=12)
ax.set_ylabel('Solar Power (MW)', fontsize=12)
ax.set_title('Ensemble vs Single Models - Last 7 Days', fontsize=14, fontweight='bold')
ax.legend(loc='upper left', fontsize=10)
ax.grid(alpha=0.3)

plt.tight_layout()
plt.savefig('../results/figures/ensemble_timeseries_comparison.png', dpi=300, bbox_inches='tight')
plt.show()

## 6. Ergebnisse speichern

In [ ]:
# CSV Export
results.to_csv('../results/metrics/ensemble_methods_comparison.csv', index=False)
print("✅ Ergebnisse gespeichert in results/metrics/ensemble_methods_comparison.csv")

# Bestes Ensemble speichern
if optimized_mae < xgb_mae:
    best_model_name = 'Optimized Ensemble'
    improvement = ((xgb_mae - optimized_mae) / xgb_mae) * 100
    print(f"\n🎉 Ensemble verbessert XGBoost um {improvement:.2f}%")
    print(f"Best Weights: XGBoost={best_weights[0]:.2f}, LSTM={best_weights[1]:.2f}, Chronos={best_weights[2]:.2f}")
else:
    print("\n⚠️ XGBoost bleibt das beste Modell")
    print(f"XGBoost übertrifft alle Ensembles")

## 7. Zusammenfassung

### Key Findings:
1. **Single Models** bleiben extrem stark (XGBoost R²=0.9825)
2. **Ensemble Methods** können leichte Verbesserungen bringen
3. **Optimized Weighting** findet beste Balance zwischen Modellen
4. **Chronos** trägt trotz schlechterer Performance zur Diversität bei

### Production Empfehlung:
- **Hauptmodell**: XGBoost (Tuned) - Beste Performance
- **Backup**: Optimized Ensemble - Robustheit durch Diversität
- **Monitoring**: Stacking Meta-Learner - Adaptive Gewichte